# 🎯 Structured Outputs with OpenAI API

Welcome! In this notebook, you'll learn how to get **structured, type-safe JSON responses** from OpenAI's API using the Responses API.

---

## 📚 What You'll Learn

By the end of this tutorial, you'll be able to:

1. **Understand response formats** - The modern way to get structured JSON from LLMs
2. **Use JSON Schema** - Define exact structure and types for your outputs
3. **Handle complex data** - Work with nested objects, arrays, and nullable fields
4. **Apply best practices** - Avoid common pitfalls and build production-ready systems
5. **Build real applications** - Create practical tools like event extractors and data parsers

---

## 💡 Prerequisites

This notebook assumes you already know:
- How to make basic OpenAI API calls
- Function calling concepts (covered in notebook 13)
- Basic JSON structure

---

Let's get started! 🚀

---

# 0️⃣ Setup

First, let's install the required libraries and configure our OpenAI API key.

In [ ]:
# Install required packages
!pip install -q openai

In [ ]:
# Import necessary libraries
import os
import json
from openai import OpenAI

## 🔑 API Key Configuration

You have two methods to provide your OpenAI API key:

**Method 1 (Recommended)**: Use Colab Secrets
1. Click the 🔑 icon in the left sidebar
2. Click "Add new secret"
3. Name: `OPENAI_API_KEY`
4. Value: Your OpenAI API key
5. Enable notebook access

**Method 2 (Fallback)**: Manual input when prompted

In [ ]:
# Configure OpenAI API key
# Method 1: Try to get API key from Colab secrets (recommended)
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Method 2: Manual input (fallback)
    from getpass import getpass
    print("💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Validate that the API key is set
if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == "":
    raise ValueError("❌ ERROR: No API key provided!")

print("✅ Authentication configured!")

# Configure which OpenAI model to use
OPENAI_MODEL = "gpt-5-nano"  # Using gpt-5-nano for cost efficiency
print(f"🤖 Selected Model: {OPENAI_MODEL}")

In [ ]:
# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ OpenAI client initialized successfully!")
print("🎓 Ready to learn about Structured Outputs!")

---

# 1️⃣ Introduction to Structured Outputs

## 🤔 Two Ways to Get Structured Data

OpenAI offers two main approaches to get structured outputs from language models:

### 1. **Function Calling** (Covered in Notebook 13)
- The model requests function execution
- You parse the function arguments
- Useful when you want the model to trigger actions

### 2. **Response Formats** (Focus of This Notebook) ✨
- The model responds **directly in structured JSON**
- No function calls - just get the data you need
- Simpler for pure data extraction tasks

---

## 🎯 Why Use Response Formats?

Response formats are perfect when you want to:
- **Extract information** from text (emails, documents, descriptions)
- **Parse unstructured data** into structured forms
- **Generate structured content** (product listings, event details, user profiles)

### 🌟 Key Benefits:

1. **Type Safety** 🛡️ - Get guaranteed data types (integers stay integers, not strings)
2. **Consistency** 📏 - Field names and structure stay the same every time
3. **Explicit Refusals** 🚫 - The model can refuse to output when appropriate
4. **Simpler Prompting** ✍️ - No need to write detailed formatting instructions
5. **Production Ready** 🏭 - Reliable enough for real applications

---

## 💡 Key Insight

**Response formats are ideal for data extraction tasks** where you don't need the model to trigger functions or actions - you just want structured data back. Think of it as asking the model to fill out a form with specific fields!

---

### ✅ Key Takeaways:

- Function calling is for triggering actions; response formats are for getting structured data
- Response formats provide type-safe, consistent JSON outputs
- Use response formats when you need to extract or generate structured information
- This approach simplifies prompting and is production-ready

---

# 2️⃣ Response Formats - Getting Structured JSON

Let's explore how to get structured JSON responses from the OpenAI Responses API. We'll start with the old way, then move to the modern, better approach!

## 2.1 The Old Way - Prompting for JSON ⚠️

### 📖 What is Prompt-Based JSON?

Before structured outputs became available, the only way to get JSON responses from language models was to simply ask for it in your prompt. You would write something like "return the result as JSON" or "extract this information in JSON format with fields X, Y, and Z" and hope the model would comply. While this approach often worked, it came with significant limitations that made it unsuitable for production applications.

### 🚨 The Core Problem: No Guaranteed Structure

The fundamental issue with prompt-based JSON is that **you have no control over the output structure**. The model tries its best to follow your instructions, but it interprets them probabilistically. Each time you make the same request, the model might make slightly different decisions about how to format the response, leading to inconsistent outputs.

### 💔 Major Limitations and Problems

**1. Inconsistent Field Names**

Even when you explicitly specify field names in your prompt, the model might use variations. For example, if you ask for a "price" field, one response might use `"price"`, another might use `"cost"`, and yet another might use `"product_price"` or `"unit_price"`. This inconsistency breaks your parsing code, as you can't reliably access the data without checking multiple possible field names.

**2. Unpredictable Data Types**

Type consistency is another major problem. A price value might come back as a number (`49.99`) in one response and as a string (`"$49.99"` or `"49.99"`) in another. This makes it impossible to perform calculations or comparisons without extensive type checking and conversion logic. The same applies to booleans (might get `true`, `"true"`, `"yes"`, or `1`) and integers (might get `2024` or `"2024"`).

**3. Missing or Extra Fields**

The model might decide to omit fields you requested if it thinks the information isn't available or isn't important. Conversely, it might add extra fields that you didn't ask for, thinking they would be helpful. This variability means you can't rely on a consistent structure when processing the responses.

**4. Structural Variations**

Beyond individual fields, the overall structure might change. Sometimes the model returns a flat object, other times it nests related fields into sub-objects. For example, you might get `{"price": 49.99, "currency": "USD"}` in one response and `{"price": {"amount": 49.99, "currency": "USD"}}` in another.

**5. Invalid JSON**

While modern models are generally good at producing valid JSON syntax, there's still no guarantee. The model might occasionally produce malformed JSON that fails to parse, especially with complex nested structures or when handling special characters.

### ⚙️ Why This Happens

These inconsistencies aren't bugs—they're inherent to how language models work. Models generate text probabilistically, token by token. Without explicit constraints, the model makes independent decisions about field names, types, and structure based on its training and the context. Even with detailed prompting, you're relying on the model's interpretation rather than enforcing rules.

### 📉 Impact on Production Systems

For production applications, these limitations are deal-breakers:

- **Unreliable parsing**: Your code needs complex error handling to account for all possible variations
- **Fragile integrations**: APIs and databases expect consistent data structures
- **Difficult debugging**: When something breaks, you need to handle countless edge cases
- **Maintenance burden**: As your prompts evolve, you might introduce new variations
- **No type safety**: You can't leverage static type checking or validation libraries effectively

In practice, developers would resort to extensive post-processing: checking multiple field name variations, converting string numbers to actual numbers, validating that required fields exist, and handling unexpected structures. This adds complexity, increases the chance of bugs, and makes the system harder to maintain.

### ✅ Key Takeaway

While prompt-based JSON works for quick prototypes or one-off tasks, it's fundamentally unreliable for production use. The lack of guaranteed structure means you're constantly fighting against variability instead of building on a solid foundation. This is precisely why structured outputs with JSON Schema were introduced—to give developers the control and consistency they need for real-world applications.

### 🔍 Problems Observed:

If you ran the cell above multiple times, you likely saw:

❌ **Inconsistent field names**:
- Run 1: `"price": 49.99`
- Run 2: `"cost": "$49.99"`
- Run 3: `"product_price": 49.99`

❌ **Inconsistent data types**:
- Sometimes price is a number: `49.99`
- Sometimes price is a string: `"$49.99"`
- Your parsing code breaks! 💥

❌ **Unreliable for production**:
- Can't write stable code when the structure keeps changing
- Need complex error handling for every possible variation
- Not scalable for real applications

### ✅ Key Takeaways:

- Prompt-based JSON guarantees neither valid JSON syntax nor consistent structure
- Field names, data types, and structure can vary between calls
- This makes it unreliable for production applications
- You need explicit schema control - which brings us to the modern way!

## 2.2 The Modern Way - `json_schema` with Responses API ✨

### 🎯 The Solution: Structured Outputs with JSON Schema

Because of these limitations with prompt-based JSON, OpenAI introduced **structured outputs** using JSON Schema. This feature fundamentally changes the game by allowing you to define an explicit schema that the model must follow. Instead of hoping the model interprets your prompt correctly, you provide a formal specification that guarantees consistency.

With structured outputs, you define exactly what fields you want, their data types, which fields are required, and even constraints like enums for fixed choices. The model then generates outputs that strictly conform to your schema—every single time. This transforms language models from unpredictable text generators into reliable structured data extractors that you can build production systems on.

### 🔑 The Responses API Syntax

The Responses API uses a specific syntax for structured outputs. Instead of the `response_format` parameter used in Chat Completions, you use the `text` parameter with a nested `format` object:

```python
text={
    "format": {
        "type": "json_schema",
        "name": "schema_name",
        "schema": your_schema,
        "strict": True
    }
}
```

This syntax is unique to the Responses API and is essential for getting structured outputs with models like `gpt-5-nano`.

### 🌟 The Magic Parameter: `strict: True`

The key to reliable structured outputs is the `strict: True` parameter. This is what transforms the API from "try to follow this schema" to "must follow this schema exactly." When you set `strict: True`:

- **Field names are guaranteed** - Always exactly as you specified, never variations
- **Data types are enforced** - Numbers are always numbers, strings are always strings, booleans are always booleans
- **Required fields are present** - Every field you mark as required will be in the response
- **No unexpected fields** - When using `additionalProperties: false`, only your defined fields appear
- **Structure is consistent** - The same schema produces the same structure every time

Without `strict: True`, you're back to the probabilistic behavior of the old way. With it, you get deterministic, type-safe outputs.

### 💡 Key Benefits Over Prompt-Based JSON

Let's compare what structured outputs give you:

| Aspect | Prompt-Based JSON ❌ | Structured Outputs ✅ |
|--------|---------------------|---------------------|
| Field names | Vary between calls | Always consistent |
| Data types | Unpredictable | Guaranteed |
| Required fields | May be missing | Always present |
| Extra fields | May appear | Prevented with additionalProperties: false |
| Structure | Can vary | Fixed by schema |
| Type safety | Impossible | Full support |
| Production ready | No | Yes |

### 📋 How It Works

When you use structured outputs with JSON Schema:

1. **Define your schema** - Specify the exact structure you want using JSON Schema format
2. **Set strict mode** - Include `strict: True` to enforce the schema
3. **Make the API call** - The model generates content that conforms to your schema
4. **Parse with confidence** - You can trust the structure and types of the response

The model understands your schema and generates tokens that are guaranteed to produce valid output matching your specification. This isn't post-processing or validation—it's built into the generation process itself.

### 🚀 Ready for Production

With structured outputs, you can:
- Write parsing code once and trust it will always work
- Integrate directly with databases and APIs
- Use static type checking in your application
- Validate data using your schema
- Build reliable, maintainable systems

Let's see this in action with a practical example!

In [ ]:
# Example: Extract product information using json_schema
# ✅ This is the MODERN way - notice the consistency!

product_description = """
The TechPro Wireless Mouse is a premium ergonomic mouse perfect for office work. 
It's priced at $49.99 and falls under the Electronics category. 
Features include 2400 DPI precision and 18-month battery life.
"""

print("🔍 Extracting product information using json_schema...\n")

# Define our schema - exactly what we want!
product_schema = {
    "type": "object",  # The response is a JSON object
    "properties": {  # Define each field
        "name": {
            "type": "string",  # Product name as text
            "description": "The name of the product"
        },
        "price": {
            "type": "number",  # Price as a number (not string!)
            "description": "The price in USD as a number"
        },
        "category": {
            "type": "string",  # Category as text
            "description": "The product category"
        },
        "description": {
            "type": "string",  # Description as text
            "description": "Brief product description"
        }
    },
    "required": ["name", "price", "category", "description"],  # All fields required
    "additionalProperties": False  # No extra fields allowed
}

# Make API call with json_schema format using Responses API syntax
# IMPORTANT: Responses API uses text.format, not response_format!
response = client.responses.create(
    model=OPENAI_MODEL,
    input=f"Extract product information: {product_description}",
    text={  # Use text parameter (not response_format)
        "format": {  # Nested format object
            "type": "json_schema",  # Type is json_schema
            "name": "product_extraction",  # Schema name
            "schema": product_schema,  # Our defined schema
            "strict": True  # 🔑 THE MAGIC PARAMETER - enforce schema strictly!
        }
    }
)

# Parse the JSON response
product_data = json.loads(response.output_text)

print("📦 Extracted Product Data:")
print(json.dumps(product_data, indent=2))

print("\n✅ Run this cell multiple times - the output is ALWAYS consistent!")
print("   ✓ Same field names every time")
print("   ✓ Price is always a number, not a string")
print("   ✓ All required fields are present")
print("   ✓ No unexpected extra fields")

### 🎉 Notice the Difference!

✅ **Consistent structure**:
```json
{
  "name": "TechPro Wireless Mouse",
  "price": 49.99,
  "category": "Electronics",
  "description": "Premium ergonomic mouse perfect for office work"
}
```

Run it 100 times - you'll get the **exact same structure** every single time! 🎯

### 🔑 Responses API Syntax Recap:

```python
response = client.responses.create(
    model="gpt-5-nano",
    input="your prompt",
    text={                          # Use 'text' parameter
        "format": {                 # Nested 'format' object
            "type": "json_schema",  # Type is json_schema
            "name": "schema_name",  # Give your schema a name
            "schema": {...},        # Your JSON schema
            "strict": True          # Always use strict mode!
        }
    }
)
```

### 💡 Key Point:

**Always use `strict: True` with `json_schema`** - this is what makes structured outputs reliable. Without it, you're back to the inconsistency problems!

### ✅ Key Takeaways:

- Responses API uses `text={"format": {...}}` instead of `response_format`
- `json_schema` with `strict: True` gives you full control over output structure
- Define exactly what fields you want, their types, and requirements
- The model follows your schema precisely - no surprises
- This is production-ready and scalable for real applications
- Always use `additionalProperties: False` to prevent unexpected fields

## 📝 Exercise 1: Extract Book Information

Now it's your turn! Modify the code above to extract book information with these fields:
- **title** (string) - The book title
- **author** (string) - The author's name
- **year_published** (integer) - Year as a number
- **genres** (array of strings) - List of genre tags

### 📚 Test Input:

```
The book "1984" was written by George Orwell and published in 1949. 
It's a dystopian fiction novel that also falls under political fiction and science fiction genres.
```

### 🎯 Expected Output Structure:

```json
{
  "title": "1984",
  "author": "George Orwell",
  "year_published": 1949,
  "genres": ["dystopian fiction", "political fiction", "science fiction"]
}
```

### 💡 Hints:
- Use `"type": "integer"` for the year
- Use `"type": "array"` with `"items": {"type": "string"}` for the genres
- Don't forget `"strict": True` and `"additionalProperties": False`
- Remember to use `text={"format": {...}}` syntax!

In [ ]:
# YOUR CODE HERE
# Define the book schema and extract information from the test input

book_description = """
The book "1984" was written by George Orwell and published in 1949. 
It's a dystopian fiction novel that also falls under political fiction and science fiction genres.
"""

# Define your book_schema here
book_schema = {
    # TODO: Fill in the schema
}

# Make the API call
# TODO: Complete the API call with json_schema format using text parameter

## 2.3 Understanding JSON Schema Basics 📚

### 🔤 Core Data Types

JSON Schema supports several fundamental data types. Understanding which to use is crucial for reliable structured outputs!

#### 1. **`string`** - Text data
- Names, descriptions, URLs, emails, etc.
- Example: `"John Doe"`, `"hello@example.com"`

#### 2. **`integer`** - Whole numbers
- Years, counts, IDs, quantities
- Example: `2024`, `42`, `-5`
- ⚠️ Use this, not `number`, when you need whole numbers only

#### 3. **`number`** - Decimal numbers
- Prices, ratings, measurements, percentages
- Example: `49.99`, `3.14`, `-2.5`
- Can also represent integers, but `integer` is more precise

#### 4. **`boolean`** - True/false values
- Flags, status indicators, yes/no questions
- Example: `true`, `false`

#### 5. **`array`** - Lists of items
- Collections of things (tags, names, values)
- Must specify what type of items: `"items": {"type": "string"}`
- Example: `["red", "green", "blue"]`

#### 6. **`object`** - Nested structures
- Complex data with its own fields
- Example: `{"street": "123 Main St", "city": "Boston"}`

---

### 🔐 Required Fields

The `required` array specifies which fields **must** be present:

```json
"required": ["name", "email", "age"]
```

- Listed fields cannot be omitted
- Other fields are optional
- Use this to enforce data completeness

---

### 🚫 Additional Properties

The `additionalProperties` setting controls extra fields:

```json
"additionalProperties": false
```

- `false` - Only schema-defined fields allowed (recommended!)
- `true` - Model can add extra fields
- Always set to `false` for predictable outputs

---

Let's see all these types in action!

In [ ]:
# Example: Extract movie information demonstrating all data types

movie_description = """
The movie "Inception" was released in 2010 and belongs to the Science Fiction genre. 
It has an IMDb rating of 8.8 out of 10 and runs for 148 minutes. 
The film is available on Netflix streaming.
"""

print("🎬 Extracting movie information with various data types...\n")

# Define schema demonstrating different data types
movie_schema = {
    "type": "object",
    "properties": {
        "title": {
            "type": "string",  # Text - movie title
            "description": "The movie title"
        },
        "year": {
            "type": "integer",  # Whole number - release year
            "description": "Release year as an integer"
        },
        "genre": {
            "type": "string",  # Text - genre category
            "description": "Primary genre"
        },
        "rating": {
            "type": "number",  # Decimal number - rating score
            "description": "IMDb rating as a decimal number"
        },
        "runtime_minutes": {
            "type": "integer",  # Whole number - duration in minutes
            "description": "Runtime in minutes as an integer"
        },
        "streaming_available": {
            "type": "boolean",  # True/false - availability flag
            "description": "Whether the movie is available on streaming platforms"
        }
    },
    "required": ["title", "year", "genre", "rating", "runtime_minutes", "streaming_available"],
    "additionalProperties": False  # No extra fields allowed
}

# Make API call using Responses API syntax
response = client.responses.create(
    model=OPENAI_MODEL,
    input=f"Extract movie information: {movie_description}",
    text={
        "format": {
            "type": "json_schema",
            "name": "movie_extraction",
            "schema": movie_schema,
            "strict": True  # Enforce schema strictly
        }
    }
)

# Parse and display
movie_data = json.loads(response.output_text)

print("🎬 Extracted Movie Data:")
print(json.dumps(movie_data, indent=2))

# Show the types
print("\n📊 Data Types Verification:")
print(f"   title type: {type(movie_data['title']).__name__} (string) ✓")
print(f"   year type: {type(movie_data['year']).__name__} (integer) ✓")
print(f"   genre type: {type(movie_data['genre']).__name__} (string) ✓")
print(f"   rating type: {type(movie_data['rating']).__name__} (number/float) ✓")
print(f"   runtime_minutes type: {type(movie_data['runtime_minutes']).__name__} (integer) ✓")
print(f"   streaming_available type: {type(movie_data['streaming_available']).__name__} (boolean) ✓")

### 🎯 Type Selection Guide

| Data | Use This Type | Why |
|------|---------------|-----|
| Names, descriptions, URLs | `string` | Text information |
| Years, counts, IDs | `integer` | Whole numbers only |
| Prices, ratings, percentages | `number` | May have decimals |
| Yes/no, true/false, flags | `boolean` | Binary states |
| Lists of items | `array` | Multiple values |
| Complex nested data | `object` | Structured information |

### ✅ Key Takeaways:

- Choose the most specific type that fits your data
- Use `integer` for whole numbers, `number` for decimals
- Always specify `required` fields to ensure data completeness
- Set `additionalProperties: false` to prevent unexpected fields
- Add descriptions to fields for clarity (helps the model understand your intent)

---

# 3️⃣ Practical Example - Event Information Extractor

## 📅 Scenario: Building a Calendar Assistant

Imagine you're building a smart calendar assistant that reads event descriptions from emails, messages, or meeting notes and automatically creates calendar entries. Users can write naturally about their events, and your system extracts all the structured details needed!

### 🎯 The Goal:

Extract structured event information from natural language descriptions:
- Event title and description
- Date and time details
- Location (physical or virtual)
- Attendee list
- Meeting links for virtual events

### 💡 Why This Matters:

Real-world data is messy! People describe events in different ways:
- "Let's meet tomorrow at 3pm for an hour"
- "Conference call on Zoom next Tuesday, link in email"
- "Team lunch at the office cafeteria, bring everyone"

Your system needs to handle all these variations and extract consistent, structured data.

Let's build it! 🚀

In [ ]:
# Practical Example: Event Information Extractor

# Sample event description (natural language)
event_text = """
We have a quarterly planning meeting scheduled for March 15, 2024, starting at 2:00 PM. 
The session will last approximately 90 minutes and will be held virtually via Zoom. 
The meeting link is https://zoom.us/j/123456789. 
Key attendees include Sarah Chen, Mike Rodriguez, and Jennifer Liu. 
We'll be discussing Q2 goals, budget allocation, and team restructuring plans.
"""

print("📅 Extracting event information...\n")

# Define comprehensive event schema
event_schema = {
    "type": "object",
    "properties": {
        # Basic event information
        "event_title": {
            "type": "string",
            "description": "The title or name of the event"
        },
        "description": {
            "type": "string",
            "description": "Brief description of what will be discussed or done"
        },
        
        # Date and time information
        "date": {
            "type": "string",
            "description": "Date in YYYY-MM-DD format"
        },
        "start_time": {
            "type": "string",
            "description": "Start time in HH:MM format (24-hour)"
        },
        "duration_minutes": {
            "type": "integer",  # Integer for whole number of minutes
            "description": "Duration in minutes"
        },
        
        # Location information
        "location": {
            "type": "string",
            "description": "Physical location or 'Virtual' if online"
        },
        "is_virtual": {
            "type": "boolean",  # Boolean for true/false flag
            "description": "Whether the meeting is virtual/online"
        },
        
        # Nullable field example: meeting link might not always exist
        "meeting_link": {
            "type": ["string", "null"],  # Can be string OR null
            "description": "Virtual meeting link if applicable, null otherwise"
        },
        
        # Array of attendees
        "attendees": {
            "type": "array",  # Array type for lists
            "items": {  # Define what each item in the array should be
                "type": "string"
            },
            "description": "List of attendee names"
        }
    },
    # All fields are required
    "required": [
        "event_title", 
        "description", 
        "date", 
        "start_time", 
        "duration_minutes",
        "location", 
        "is_virtual", 
        "meeting_link", 
        "attendees"
    ],
    "additionalProperties": False  # No extra fields
}

# Extract event information using Responses API
response = client.responses.create(
    model=OPENAI_MODEL,
    input=f"Extract structured event information from this description: {event_text}",
    text={
        "format": {
            "type": "json_schema",
            "name": "event_extraction",
            "schema": event_schema,
            "strict": True
        }
    }
)

# Parse the result
event_data = json.loads(response.output_text)

# Display the extracted information
print("📋 Extracted Event Details:")
print("=" * 60)
print(json.dumps(event_data, indent=2))
print("=" * 60)

# Demonstrate how you'd use this in a real application
print("\n📱 Ready for Calendar Integration:")
print(f"✓ Event: {event_data['event_title']}")
print(f"✓ When: {event_data['date']} at {event_data['start_time']} ({event_data['duration_minutes']} min)")
print(f"✓ Where: {event_data['location']} {'(Virtual)' if event_data['is_virtual'] else '(In-person)'}")
if event_data['meeting_link']:
    print(f"✓ Link: {event_data['meeting_link']}")
print(f"✓ Attendees: {', '.join(event_data['attendees'])}")
print(f"\n💡 This structured data can now be sent directly to any calendar API!")

### 🔍 Key Schema Features Demonstrated:

#### 1. **Nullable Fields** 🔄
```json
"meeting_link": {
    "type": ["string", "null"],
    "description": "Virtual meeting link if applicable, null otherwise"
}
```
- Use `["string", "null"]` when a field might not have a value
- Perfect for optional data like meeting links, middle names, or phone extensions
- The model can set it to `null` instead of forcing a value

#### 2. **Arrays of Simple Values** 📝
```json
"attendees": {
    "type": "array",
    "items": {"type": "string"},
    "description": "List of attendee names"
}
```
- Use for lists of strings, numbers, or booleans
- Must specify what type the items are

#### 3. **Specific Formats** 📅
- Date: `"YYYY-MM-DD format"` in description guides the model
- Time: `"HH:MM format (24-hour)"` ensures consistency
- Descriptions help the model understand expected formats

### 💡 Real-World Application:

This structured output can be directly sent to calendar APIs like:
- Google Calendar API
- Microsoft Outlook API
- Apple Calendar
- Any custom calendar system

No parsing, no cleaning - just map the fields and send! 🚀

### ✅ Key Takeaways:

- Use nullable types `["type", "null"]` for optional fields
- Arrays need `items` to specify what they contain
- Field descriptions guide the model on format expectations
- Structured outputs integrate seamlessly with other APIs
- Real-world scenarios require handling missing or optional data

## 📝 Exercise 2: Enhance the Event Schema

Now enhance the event extractor by adding three new fields to handle more details:

### 🎯 New Fields to Add:

1. **requires_preparation** (boolean)
   - Whether the event requires advance preparation
   - Example: "Please review the Q1 report before the meeting" → `true`

2. **reminder_minutes_before** (integer)
   - How many minutes before the event to send a reminder
   - Example: "Set a reminder 30 minutes before" → `30`

3. **priority_level** (enum: "low", "medium", "high")
   - The importance/urgency of the event
   - Use an enum to restrict values

### 📚 Test Input:

```
High-priority client presentation on March 20, 2024 at 10:00 AM, lasting 2 hours. 
This is a critical meeting at the downtown office, Conference Room A. 
Team members should review the proposal deck beforehand. 
Set a reminder 15 minutes before. 
Attendees: Alex Kim, Jordan Park, and Taylor Morgan.
```

### 💡 Hints:
- For enums, use: `"enum": ["low", "medium", "high"]`
- Add descriptions to help the model understand priority levels
- The model should infer `requires_preparation: true` from "review beforehand"
- Don't forget to add new fields to the `required` array!
- Use `text={"format": {...}}` syntax for Responses API

In [ ]:
# YOUR CODE HERE
# Enhance the event schema with the three new fields

enhanced_event_text = """
High-priority client presentation on March 20, 2024 at 10:00 AM, lasting 2 hours. 
This is a critical meeting at the downtown office, Conference Room A. 
Team members should review the proposal deck beforehand. 
Set a reminder 15 minutes before. 
Attendees: Alex Kim, Jordan Park, and Taylor Morgan.
"""

# TODO: Copy the event_schema from above and add the three new fields
enhanced_event_schema = {
    # Add your enhanced schema here
}

# TODO: Make the API call and display results using text parameter

---

# 4️⃣ Working with Nested Structures

## 🪆 Why Nested Structures?

Real-world data is often hierarchical - objects containing other objects or arrays of objects. Nested structures let you represent complex, related information accurately.

### 🎯 When You Need Nested Structures:

#### 1. **E-commerce Product with Variants** 🛍️
```
Product: T-Shirt
├─ Variants:
│  ├─ Small (Blue, $19.99, 45 in stock)
│  ├─ Medium (Red, $19.99, 30 in stock)
│  └─ Large (Black, $21.99, 50 in stock)
```
Each variant has its own color, price, and inventory - that's a nested structure!

#### 2. **Organizational Chart** 👥
```
Company: TechCorp
├─ Employees:
│  ├─ CEO (name, hire_date, salary)
│  │  └─ Reports: [list of employee IDs]
│  ├─ CTO (name, hire_date, salary)
│  │  └─ Reports: [list of employee IDs]
```
Employees have their own properties and relationships - nested objects!

#### 3. **Flight Booking** ✈️
```
Booking:
├─ Flight: (airline, number, date)
├─ Passengers:
│  ├─ Passenger 1 (name, age, seat)
│  │  └─ Baggage: [checked: 2 bags, carry-on: 1 bag]
│  ├─ Passenger 2 (name, age, seat)
│  │  └─ Baggage: [checked: 1 bag, carry-on: 1 bag]
```
Each passenger has their own details and baggage - complex nested data!

### 💡 Key Concept:

**Nested structures = Objects within objects, or arrays of objects**
- Use when data has hierarchical relationships
- Use when you have collections of complex items (not just simple strings or numbers)
- Essential for modeling real-world entities accurately

Let's see how to define and work with these structures!

In [ ]:
# Example: Extract company information with nested structures

company_description = """
TechStart Inc. is a software development company founded in 2018 in the technology industry. 
The company has 150 employees and generates $25 million in annual revenue. 

The leadership team includes:
- Sarah Johnson, CEO, who has been in her role for 5 years
- Michael Chen, CTO, who has been in his role for 4 years
- Emily Rodriguez, CFO, who has been in her role for 3 years

TechStart has offices in three locations:
- San Francisco, USA (headquarters)
- Austin, USA (not headquarters)
- Toronto, Canada (not headquarters)
"""

print("🏢 Extracting company information with nested structures...\n")

# Define schema with nested structures
company_schema = {
    "type": "object",
    "properties": {
        # Simple top-level fields
        "company_name": {
            "type": "string",
            "description": "The company name"
        },
        "founded_year": {
            "type": "integer",
            "description": "Year the company was founded"
        },
        "industry": {
            "type": "string",
            "description": "Primary industry"
        },
        "employee_count": {
            "type": "integer",
            "description": "Number of employees"
        },
        "annual_revenue_millions": {
            "type": "number",
            "description": "Annual revenue in millions of dollars"
        },
        
        # NESTED STRUCTURE 1: Array of objects (leadership team)
        # This is an array where each item is a complex object with multiple fields
        "leadership": {
            "type": "array",  # It's an array...
            "items": {  # ...where each item is:
                "type": "object",  # An object (not just a string!)
                "properties": {  # With its own fields:
                    "name": {
                        "type": "string",
                        "description": "Leader's full name"
                    },
                    "title": {
                        "type": "string",
                        "description": "Job title (CEO, CTO, etc.)"
                    },
                    "years_in_role": {
                        "type": "integer",
                        "description": "Years in current position"
                    }
                },
                "required": ["name", "title", "years_in_role"],  # Required fields for each leader
                "additionalProperties": False
            },
            "description": "List of company leadership members"
        },
        
        # NESTED STRUCTURE 2: Array of objects (office locations)
        # Another array of complex objects
        "locations": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "City name"
                    },
                    "country": {
                        "type": "string",
                        "description": "Country name"
                    },
                    "is_headquarters": {
                        "type": "boolean",
                        "description": "Whether this is the headquarters location"
                    }
                },
                "required": ["city", "country", "is_headquarters"],
                "additionalProperties": False
            },
            "description": "List of office locations"
        }
    },
    "required": [
        "company_name", 
        "founded_year", 
        "industry", 
        "employee_count", 
        "annual_revenue_millions",
        "leadership", 
        "locations"
    ],
    "additionalProperties": False
}

# Extract company information using Responses API
response = client.responses.create(
    model=OPENAI_MODEL,
    input=f"Extract structured company information: {company_description}",
    text={
        "format": {
            "type": "json_schema",
            "name": "company_extraction",
            "schema": company_schema,
            "strict": True
        }
    }
)

# Parse and display
company_data = json.loads(response.output_text)

print("🏢 Extracted Company Data:")
print("=" * 70)
print(json.dumps(company_data, indent=2))
print("=" * 70)

# Demonstrate accessing nested data
print("\n📊 Accessing Nested Data:")
print(f"\n👔 Leadership Team ({len(company_data['leadership'])} members):")
for leader in company_data['leadership']:
    print(f"   • {leader['name']} - {leader['title']} ({leader['years_in_role']} years)")

print(f"\n🌍 Office Locations ({len(company_data['locations'])} locations):")
for location in company_data['locations']:
    hq_marker = "⭐ HQ" if location['is_headquarters'] else "📍"
    print(f"   {hq_marker} {location['city']}, {location['country']}")

print("\n💡 Notice how we can easily loop through nested arrays and access their fields!")

### 🔍 Understanding Nested Structures:

#### 📦 Array of Objects Pattern:

```json
{
  "type": "array",          // It's an array
  "items": {                 // Each item in the array is:
    "type": "object",        // An object
    "properties": {          // With these fields:
      "field1": {"type": "string"},
      "field2": {"type": "integer"}
    },
    "required": ["field1", "field2"],
    "additionalProperties": false
  }
}
```

#### 🎯 Key Points:

1. **Array of Simple Values** (strings, numbers):
   ```json
   "tags": {
     "type": "array",
     "items": {"type": "string"}  // Just specify the type
   }
   ```
   Result: `["tech", "startup", "AI"]`

2. **Array of Objects** (complex items):
   ```json
   "employees": {
     "type": "array",
     "items": {
       "type": "object",         // Each item is an object
       "properties": {...},       // Define its structure
       "required": [...]
     }
   }
   ```
   Result: `[{"name": "Alice", "role": "Dev"}, {"name": "Bob", "role": "PM"}]`

3. **Required Fields Apply to Each Object**:
   - Set `required` inside the `items` definition
   - Every object in the array must have those fields

4. **Use `additionalProperties: false`** at both levels:
   - Top level (main object)
   - Nested level (objects within arrays)

### 💡 Real-World Benefit:

With nested structures, you can model complex entities accurately:
- ✅ Each leader has consistent fields (name, title, years)
- ✅ Easy to loop through and access
- ✅ Type-safe - years is always an integer
- ✅ No parsing headaches!

### ✅ Key Takeaways:

- Nested structures model real-world hierarchical data
- Use `"type": "array"` with `"items": {"type": "object"}` for arrays of complex items
- Define `properties` and `required` within the `items` definition
- Each object in the array follows the same schema
- Set `additionalProperties: false` at all levels for strictness
- Nested data is easy to work with in code (loops, accessing fields)

---

# 5️⃣ Best Practices and Common Pitfalls

Learn how to write production-ready schemas and avoid common mistakes!

## ✅ Best Practices

### 1. Always Use `strict: True` 🔒

This is the **most important parameter** for reliable structured outputs!

**✅ Correct:**
```python
response = client.responses.create(
    model=OPENAI_MODEL,
    input=prompt,
    text={
        "format": {
            "type": "json_schema",
            "name": "my_schema",
            "schema": your_schema,
            "strict": True  # ✅ Always include this!
        }
    }
)
```

**❌ Wrong:**
```python
# Missing strict: True
text={
    "format": {
        "type": "json_schema",
        "name": "my_schema",
        "schema": your_schema  # ❌ Not enforced strictly!
    }
}
```

**Why it matters:** Without `strict: True`, the model may deviate from your schema. With it, you get **guaranteed consistency**.

---

### 2. Use Enums for Fixed Choices 🎯

When a field should only accept specific values, use enums!

**✅ Correct:**
```python
"status": {
    "type": "string",
    "enum": ["pending", "approved", "rejected"],  # Only these values allowed
    "description": "The approval status"
}
```
Result: Always one of the three values, never "Pending", "APPROVED", or misspellings.

**❌ Wrong:**
```python
"status": {
    "type": "string",  # No enum - any string allowed
    "description": "Status (should be pending, approved, or rejected)"
}
```
Result: Might get "Pending", "approved", "denied", "Approved" - inconsistent!

**When to use enums:**
- Priority levels (low, medium, high)
- Status values (active, inactive, pending)
- Categories (fiction, non-fiction, biography)
- Any field with a fixed set of options

---

### 3. Add Descriptions to Fields 📝

Descriptions help the model understand what you want and improve extraction accuracy!

**✅ Correct:**
```python
"price": {
    "type": "number",
    "description": "Product price in USD as a decimal number (e.g., 29.99)"
}
```

**❌ Less Clear:**
```python
"price": {
    "type": "number"  # No description - less guidance for the model
}
```

**Good description patterns:**
- Specify format: `"Date in YYYY-MM-DD format"`
- Give examples: `"Duration in minutes (e.g., 90)"`
- Clarify ambiguity: `"The person's age in years as an integer"`
- Explain enums: `"Priority: low (minor), medium (normal), high (urgent)"`

---

### 4. Set `additionalProperties` to `False` 🚫

Prevent the model from adding unexpected fields!

**✅ Correct:**
```python
{
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "integer"}
    },
    "required": ["name", "age"],
    "additionalProperties": False  # ✅ Only name and age allowed
}
```

**❌ Wrong:**
```python
{
    "type": "object",
    "properties": {...},
    "required": [...]
    # Missing additionalProperties: False
}
```
Result: Model might add extra fields like `"nickname"`, `"email"`, etc.

---

### 5. Use Appropriate Data Types 🎲

Choose the most specific type that fits your data!

**✅ Correct Choices:**
```python
# Years, quantities, counts
"year": {"type": "integer"}  # 2024, not "2024" or 2024.0

# Money, ratings, measurements
"price": {"type": "number"}  # 29.99, can have decimals

# Yes/no, true/false
"in_stock": {"type": "boolean"}  # true or false, not "yes"/"no"

# Lists of items
"tags": {
    "type": "array",
    "items": {"type": "string"}  # Specify item type!
}
```

**❌ Common Mistakes:**
```python
# Using string for numbers
"price": {"type": "string"}  # ❌ Forces "29.99" instead of 29.99

# Using number for years
"year": {"type": "number"}  # ❌ Could be 2024.5, use integer!

# Using string for boolean
"active": {"type": "string"}  # ❌ Gets "true"/"yes"/"active", use boolean!
```

---

### 6. Handle Nullable Fields Properly ⚪

When a field might not have a value, make it nullable!

**✅ Correct:**
```python
"middle_name": {
    "type": ["string", "null"],  # Can be string OR null
    "description": "Middle name if available, null otherwise"
}
```
Result: Can be `"Lee"` or `null` - both valid!

**❌ Wrong:**
```python
# Option 1: Not in required but no null type
"middle_name": {"type": "string"}  # ❌ Still required if in required array!

# Option 2: Making it optional by omitting from required
"required": ["first_name", "last_name"]  # middle_name not listed
# ❌ Field might be missing entirely from the JSON
```

**When to use nullable:**
- Optional contact info (phone, fax, ext)
- Virtual meeting links (only for online events)
- Middle names, suffixes, prefixes
- Any field that genuinely might not exist

**Best Practice:** Use `["type", "null"]` and include in `required` array. This way the field is always present but can be `null`.

---

### ✅ Summary: Best Practices Checklist

Before deploying your schema, verify:
- ✅ `strict: True` is set
- ✅ Enums defined for fixed choices
- ✅ All fields have descriptions
- ✅ `additionalProperties: False` at all levels
- ✅ Most specific data types chosen
- ✅ Nullable types used where appropriate
- ✅ Required fields properly listed
- ✅ Nested structures have their own `required` and `additionalProperties`
- ✅ Using `text={"format": {...}}` syntax for Responses API

## ⚠️ Common Pitfalls

Learn from these common mistakes to save debugging time!

### ❌ Pitfall 1: Forgetting `strict: True`

This is the #1 mistake! Without `strict: True`, you lose all consistency guarantees.

**Wrong:**
```python
text={
    "format": {
        "type": "json_schema",
        "name": "product_schema",
        "schema": product_schema
        # ❌ Missing strict: True!
    }
}
```

**Result:** Field names might vary, types might be wrong, structure might differ between calls.

**Correct:**
```python
text={
    "format": {
        "type": "json_schema",
        "name": "product_schema",
        "schema": product_schema,
        "strict": True  # ✅ Always include!
    }
}
```

**💡 Rule:** If you're using `json_schema`, ALWAYS set `strict: True`. No exceptions!

---

### ❌ Pitfall 2: Using Wrong Data Types

Choosing the wrong type causes parsing errors and logic bugs.

**Common Mistake #1: String for Prices**
```python
# ❌ Wrong
"price": {
    "type": "string",
    "description": "Product price"
}
```
Result: `"price": "$49.99"` or `"price": "49.99"`
Problem: Can't do math! Need to parse, remove $, convert to float.

```python
# ✅ Correct
"price": {
    "type": "number",
    "description": "Product price in USD as a decimal number"
}
```
Result: `"price": 49.99`
Benefit: Ready for calculations, comparisons, sorting!

**💡 Rule:** Choose the most specific type. Integer over number for whole numbers, boolean over string for yes/no.

---

### ❌ Pitfall 3: Not Specifying Required Fields

Missing required fields leads to incomplete data.

**Wrong:**
```python
{
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string"},
        "phone": {"type": "string"}
    }
    # ❌ No required array - all fields optional!
}
```
Result: Model might omit fields. Some responses have all fields, others are missing email or phone.

**Correct:**
```python
{
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string"},
        "phone": {"type": ["string", "null"]}  # Nullable if optional
    },
    "required": ["name", "email", "phone"]  # ✅ All must be present
}
```
Result: Every response has all three fields. Phone can be `null` if not available.

**💡 Rule:** Always define `required` array. Make fields nullable if they can be missing, but keep them in required.

---

### ❌ Pitfall 4: Overcomplicating Schemas

Don't try to capture every possible detail. Keep it simple!

**Overcomplicated (❌ Too Much):**
```python
"person": {
    "type": "object",
    "properties": {
        "first_name": {"type": "string"},
        "middle_name": {"type": ["string", "null"]},
        "last_name": {"type": "string"},
        "suffix": {"type": ["string", "null"]},
        "preferred_name": {"type": ["string", "null"]},
        "maiden_name": {"type": ["string", "null"]},
        "pronunciation": {"type": ["string", "null"]},
        # ... even more fields ...
    }
}
# ❌ Too detailed - hard to extract reliably from typical text!
```

**Simple (✅ Just Right):**
```python
"person": {
    "type": "object",
    "properties": {
        "full_name": {"type": "string"},
        "email": {"type": "string"}
    },
    "required": ["full_name", "email"]
}
# ✅ Simple, easy to extract from most text!
```

**💡 Rule:** 
- Start simple, add complexity only when needed
- If the text rarely contains a field, don't require it
- 5-10 fields is usually the sweet spot
- More fields = harder to extract accurately

---

### ❌ Pitfall 5: Not Handling Missing Data

Real-world text often lacks some information. Your schema must handle this!

**Wrong Approach:**
```python
# Text: "The book was published in 1984"
# Schema requires: title, author, year

{
    "properties": {
        "title": {"type": "string"},
        "author": {"type": "string"},  # ❌ Required but not in text!
        "year": {"type": "integer"}
    },
    "required": ["title", "author", "year"]
}
```
Result: Model might hallucinate an author or fail!

**Correct Approach:**
```python
{
    "properties": {
        "title": {"type": ["string", "null"]},
        "author": {"type": ["string", "null"]},  # ✅ Can be null
        "year": {"type": ["integer", "null"]}  # ✅ Can be null
    },
    "required": ["title", "author", "year"]  # Always present but can be null
}
```
Result: `{"title": null, "author": null, "year": 1984}` - honest about missing data!

**💡 Rules:**
- Use `["type", "null"]` for fields that might be missing
- Keep fields in `required` array but make them nullable
- This gives consistent structure while allowing missing data
- Your code can check: `if data['author'] is not None:`

---

### ✅ Summary: Common Pitfalls Checklist

Before deploying, check:
- ✅ `strict: True` is present
- ✅ Data types are specific (integer vs number, boolean vs string)
- ✅ `required` array is defined
- ✅ Schema is simple enough for reliable extraction
- ✅ Fields use nullable types when data might be missing
- ✅ Tested with real-world messy text, not just perfect examples
- ✅ Using correct Responses API syntax: `text={"format": {...}}`

---

# 🎓 Conclusion

## 🎉 What You've Learned

Congratulations! You now know how to get reliable, type-safe structured outputs from OpenAI's Responses API. Let's recap:

### ✅ Key Concepts:

1. **Response Formats** - The modern way to get structured JSON
   - Prompt-based: Old way, inconsistent
   - `json_schema` with `strict: True`: Modern way, reliable

2. **Responses API Syntax** - The correct way to use structured outputs
   - Use `text={"format": {...}}` parameter
   - NOT `response_format` (that's for Chat Completions API)
   - Always include `"strict": True`

3. **JSON Schema Basics** - Defining structure and types
   - Core types: string, integer, number, boolean, array, object
   - Required fields and additionalProperties
   - Field descriptions for clarity

4. **Complex Structures** - Real-world data modeling
   - Arrays of simple values
   - Arrays of objects (nested structures)
   - Nullable fields for optional data
   - Enums for fixed choices

5. **Best Practices** - Production-ready schemas
   - Always use `strict: True`
   - Choose specific data types
   - Add descriptions to fields
   - Handle missing data with nullable types

6. **Common Pitfalls** - What to avoid
   - Forgetting `strict: True`
   - Wrong data types
   - Missing required fields
   - Overcomplicating schemas

---

## 🚀 Where to Go From Here

### Practice Projects:

1. **Resume Parser** - Extract structured job history from resumes
2. **Recipe Extractor** - Parse cooking recipes into structured format
3. **Email Parser** - Extract action items and deadlines from emails
4. **Invoice Parser** - Extract line items, totals, dates from invoices
5. **Product Review Analyzer** - Extract ratings, pros, cons from reviews

### Integration Ideas:

- Build a Slack bot that extracts meeting info and creates calendar events
- Create a Chrome extension that extracts product details from any e-commerce site
- Develop an email assistant that categorizes and extracts key information
- Build a document processor that converts PDFs to structured data

---

## 💡 Final Tips

### Remember:

1. **Start Simple** - Begin with basic schemas and add complexity gradually
2. **Test with Real Data** - Don't just use perfect examples
3. **Iterate** - Refine your schema based on actual results
4. **Handle Errors** - Add try/except blocks for production code
5. **Monitor Quality** - Check extraction accuracy over time

### The Golden Rules:

🔑 **Always use `strict: True`**

🔑 **Choose the most specific type**

🔑 **Set `additionalProperties: False`**

🔑 **Use nullable types for optional data**

🔑 **Keep schemas simple and focused**

🔑 **Use `text={"format": {...}}` for Responses API**

---

## 🎯 You're Ready!

You now have the knowledge to build production-ready data extraction systems using structured outputs with the Responses API. Go build something amazing! 🚀

### 📚 Additional Resources:

- [OpenAI Structured Outputs Guide](https://platform.openai.com/docs/guides/structured-outputs)
- [JSON Schema Documentation](https://json-schema.org/)
- [OpenAI API Reference](https://platform.openai.com/docs/api-reference)

---

**Happy building!** 🎉✨